In [ ]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [ ]:
fnl_df_brd_w_prd = pd.read_csv("fnl_df_brd_w_prd.csv")

import ast
for i in fnl_df_brd_w_prd.index:
    fnl_df_brd_w_prd.at[i, 'product_link'] = ast.literal_eval(fnl_df_brd_w_prd.loc[i, 'product_link'])
    fnl_df_brd_w_prd.at[i, 'product_name'] = ast.literal_eval(fnl_df_brd_w_prd.loc[i, 'product_name'])
    
fnl_df_glowpick_prd_info_w_img_src = pd.read_csv("fnl_df_glowpick_prd_info_w_img_src.csv")
fnl_df_glowpick_prd_info_w_img_src_drop_naver = pd.read_csv("fnl_df_glowpick_prd_info_w_img_src_drop_naver.csv")

In [ ]:
naver_beauty_product_info = pd.read_csv("naver_beauty_product_info.csv")
brand_key_added = pd.read_csv("brand_key_added.csv")

brand_naver_w_glowpick = list(brand_key_added.loc[brand_key_added['glowpick_brand_name'].notna(), 'glowpick_brand_name'])

In [ ]:
#네이버 뷰티윈도우 브랜드들의 글로우픽에서의 이름
brand_naver_w_glowpick = list(brand_key_added.loc[brand_key_added['glowpick_brand_name'].notna(), 'glowpick_brand_name'])

In [ ]:
#네이버 뷰티윈도우에 있는 브랜드의 제품 드랍

for i in tqdm(fnl_df_glowpick_prd_info_w_img_src.index):
    if fnl_df_glowpick_prd_info_w_img_src.loc[i, '브랜드명'] in brand_naver_w_glowpick:
        fnl_df_glowpick_prd_info_w_img_src_drop_naver = fnl_df_glowpick_prd_info_w_img_src.drop(i)

In [ ]:
fnl_df_glowpick_prd_info_w_img_src_drop_naver = pd.read_csv("fnl_df_glowpick_prd_info_w_img_src_drop_naver.csv", index=False)

In [ ]:
def get_img_src(prd_link):
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    
    while True:
        try:
            
            wd.get(prd_link)
            time.sleep(5)
            wd.maximize_window()

            socket.setdefaulttimeout(30)

            try:

                wd.find_element_by_xpath('//*[@id=\"default-layout\"]/div/div[1]/span/div/div[2]/div[2]/button[2]').click()
                time.sleep(5)

                html = wd.page_source
                bs = BeautifulSoup(html, 'html.parser')

                time.sleep(3)
                img_src = wd.find_element_by_xpath("/html/body/div/div/div/div/main/div/section/div[1]/div/div/img").get_attribute('src')
                img_src = re.sub("w=[0-9]*", "w=500", img_src)
                wd.close()

                return img_src

            except Exception as e:
                print(e)
                return None
                
        except urllib3.exceptions.ReadTimeoutError:
            time.sleep(5)
            
        except:
            return None

In [ ]:
for i in tqdm(list(fnl_df_glowpick_prd_info_w_img_src_drop_naver.loc[pd.isnull(fnl_df_glowpick_prd_info_w_img_src_drop_naver['이미지 src'])].index)):
    prd_link = fnl_df_glowpick_prd_info_w_img_src_drop_naver.loc[i, '제품 link']
    img_src = get_img_src(prd_link)
    if img_src != None:
        fnl_df_glowpick_prd_info_w_img_src_drop_naver.loc[i, '이미지 src'] = img_src

In [ ]:
fnl_df_glowpick_prd_info_w_img_src_drop_naver = pd.read_csv("fnl_df_glowpick_prd_info_w_img_src_drop_naver.csv")